In [13]:
from pyspark import *
from pyspark.ml.feature import *
from pyspark.sql.session import *
from pyspark.ml import *
from pyspark.sql import *
from pyspark.sql.functions import *

In [11]:
# spark session 필수 설정
spark = SparkSession.builder.appName("Test").config("spark.executor.instances", "1").config("spark.executor.cores", "1").config("spark.executor.memory", "2g").config("spark.driver.memory", "2g").getOrCreate()


In [3]:
person = spark.createDataFrame([(0, "Bill Chambers", 0, [100]),
                               (1, "Matei Zaharia", 1, [500,250,100]),
                               (2, "Michael Armbrust", 1, [250,100])])\
                               .toDF("id","name","graduate_program","spark_status")
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")])\
  .toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")])\
  .toDF("id", "status")

In [10]:
# inner join
joinExpression = person["graduate_program"] == graduateProgram['id']
person.join(graduateProgram, joinExpression).show()
# 같다
joinType = "inner"
person.join(graduateProgram, joinExpression, joinType).show()

# outer
joinType = "outer"
person.join(graduateProgram, joinExpression, joinType).show()
# left_outer
joinType = "left_outer"
graduateProgram.join(person, joinExpression, joinType).show()

# right_outer
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

# 새로운 Table 만들기
gradProgram2 = graduateProgram.union(spark.createDataFrame([
    (0, "Masters", "Duplicated Row", "Duplicated School")]))

gradProgram2.createOrReplaceTempView("gradProgram2")

# left semi 새로운 테이블을 만든다
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

# 오른쪽 있ㄴ느것만 남긴다
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

# cross
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               1|     [250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

In [11]:
# cross
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()
person.crossJoin(graduateProgram).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+--

In [3]:
csvFile = spark.read.format("csv").option("header","true").option(
"mode","FAILFAST").option("inferSchema","true").load("./data/exam/flight-data/csv/2010-summary.csv")

csvFile.show(3)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
+-----------------+-------------------+-----+
only showing top 3 rows



In [14]:
csvFile.write.format("csv").mode("overwrite").option("sep","\t").save("./data/my-tsv-file.tsv")

In [17]:
parFile = spark.read.format("parquet").load("./data/exam/flight-data/parquet/2010-summary.parquet")

In [19]:
csvFile.write.format("parquet").mode("overwrite").save("./data/my-par-file.parquet")

In [ ]:
# query = "(select empno,ename,dname from emp, dept where emp.deptno = dept.deptno) emp"
# empDF = spark.read.format("jdbc").option("url", "jdbc:oracle:thin:scott/oracle@//localhost:1521/orcl")
#     .option("dbtable", query)
#     .option("user", "scott")
#     .option("password", "oracle")
#     .option("driver", "oracle.jdbc.driver.OracleDriver")
#     .load()
# empDF.printSchema()
# empDF.show()

In [27]:
spark.read.text("/home/lab01/data/exam/flight-data/csv/2010-summary.csv")\
.selectExpr("split(value, ',')as rows").show(3,False)

+-----------------------------------------------+
|rows                                           |
+-----------------------------------------------+
|[DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME, count]|
|[United States, Romania, 1]                    |
|[United States, Ireland, 264]                  |
+-----------------------------------------------+
only showing top 3 rows

